# Black Scholes Model

Some code snippets to solve the problem classically

In [1]:
# importations
import numpy as np
from scipy.stats import norm
import math
from math import exp, sqrt

## Definition and variables

The Black-Scholes model is a mathematical model for the dynamics of a financial market containing derivative investment instruments. From the Black-Scholes equation, one can deduce the Black-Scholes formula, which gives a theoretical estimate of the price of European-style options. This formula relies on several key assumptions and uses the following variables defined in our notebook:

*   `S0`: The current stock price (initial stock price).
*   `K`: The strike price of the option.
*   `r`: The risk-free interest rate (annualized).
*   `sigma`: The volatility of the stock's returns (annualized standard deviation).
*   `T`: The time to expiration of the option (in years).

In [3]:
S0 = 1
K = 1
r = 0.02
sigma = 0.2
T = 5
n_steps = 5
n_sim = 10000

## Simulations


The price of a put and a call can be calculated using an analytical formula with the Black-Scholes model. The value of $S_0$ is the initial value of the asset price, the value of $K$ is the strike price. The value of $r$ is the risk-free interest rate and the value $\sigma$ is the volatility of the stock price. The maturity of the option in years is $T$. The price of a put and a call option can be approximated using stochastic simulations using the following formula:

## Put Option

$$\text{Put} = e^{-rT}E\left[\max(K - S_T, 0)\right] = e^{-rT} \frac{1}{M} \sum_{i=1}^{M} \max(K - S_T^{(i)}, 0)$$

## Call Option

$$\text{Call} = e^{-rT}E\left[\max(S_T - K, 0)\right] = e^{-rT} \frac{1}{M} \sum_{i=1}^{M} \max(S_T^{(i)} - K, 0)$$

where $S_T^{(i)}$ is the simulated price of the simulation $i$ and $M$ is the number of simulations.

We can simulate the value of $S_T^{(i)}$ using the following algorithm:

1. **Step 1:** generate a random number from a standard normal distribution $W^{(i)}$;

2. **Step 2:** calculate $S_T^{(i)} = \exp\left((r - 0.5\sigma^2)T + \sqrt{T}\sigma W^{(i)}\right)$;

3. **Step 3:** repeat the first and second steps $M$ times;

4. **Step 4:** approximate the value of the put price using

   $$e^{-rT} \frac{1}{M} \sum_{i=1}^{M} \max(K - S_T^{(i)}, 0)$$

   or the value of the call price using

   $$e^{-rT} \frac{1}{M} \sum_{i=1}^{M} \max(S_T^{(i)} - K, 0).$$

The values of $\sqrt{T}W^{(i)}$ can be approximated using a random walk. Also, a high value of $M$ will generate better approximations of the put and call values.

In [2]:
def black_Scholes_Put_Simulation(S0, K, r,  sigma, T, n_sim):
    mean = (r - 0.5 * sigma ** 2) * T
    std = sigma * np.sqrt(T)
    rng = np.random
    z = rng.normal(loc = mean, scale = std, size = n_sim)
    ST = S0 * np.exp(z)
    payoff = np.maximum(K - ST, 0.0)
    price = np.exp(-r * T) * payoff.mean()
    return float(price)

def black_Scholes_Call_Simulation(S0, K, r, sigma, T, n_sim):
    mean = (r - 0.5 * sigma ** 2) * T
    std = sigma * np.sqrt(T)
    rng = np.random
    z = rng.normal(loc = mean, scale = std, size = n_sim)
    ST = S0 * np.exp(z)
    payoff = np.maximum(ST - K, 0.0)
    price = np.exp(-r * T) * payoff.mean()
    return float(price)

In [6]:
put_simulation = black_Scholes_Put_Simulation(S0, K, r,  sigma, T, n_sim)
call_simulation = black_Scholes_Call_Simulation(S0, K, r,  sigma, T, n_sim)
print(f"Put Simulation = {put_simulation}")
print(f"Call Simulation = {call_simulation}")

Put Simulation = 0.12289030475464921
Call Simulation = 0.21783192837291862


## Binomial Tree

In [8]:
def black_scholes_call_binomial_tree(S0, K, r, sigma, T, n_steps):
    delta_t = T / n_steps
    u = np.exp(np.sqrt(delta_t) * sigma)
    d = np.exp(-np.sqrt(delta_t) * sigma)
    p = (np.exp(delta_t * r) - d) / (u - d)

    n_rows = 2 * n_steps + 1
    n_rows_mid = (n_rows + 1) // 2
    tree = np.zeros((n_rows, n_steps + 1))
    tree[n_rows_mid - 1, 0] = S0

    for i in range(1, n_steps + 1):
        old_idx_non_zero = np.where(tree[:, i - 1] != 0)[0]
        new_idx_non_zero = np.unique(np.concatenate((old_idx_non_zero + 1, old_idx_non_zero - 1)))
        new_idx_non_zero = new_idx_non_zero[(new_idx_non_zero >= 0) & (new_idx_non_zero < n_rows)]
        new_S_val = np.unique(S0 * (u ** (new_idx_non_zero - (n_rows_mid - 1))))
        tree[new_idx_non_zero, i] = new_S_val

    x = np.arange(0, n_steps + 1)
    vec_prob_payoff = np.array([math.comb(n_steps, k) * (1 - p) ** k * p ** (n_steps - k) for k in x])
    bool_payoff = np.array([True if i % 2 == 0 else False for i in range(2 * n_steps)])
    bool_payoff = np.append(bool_payoff, True)

    vec_payoff = np.maximum(tree[bool_payoff, n_steps] - K, 0)
    call_price_binomial_tree = np.exp(-T * r) * np.sum(vec_payoff * vec_prob_payoff)

    return {
        "mat_Binomial_Tree": tree,
        "vec_Payoff": vec_payoff,
        "vec_Prob_Payoff": vec_prob_payoff,
        "call_Price_Binomial_Tree": call_price_binomial_tree,
        "p": p,
        "d": d,
        "delta_t": delta_t
    }

def black_scholes_put_binomial_tree(S0, K, r, sigma, T, n_steps):
    delta_t = T / n_steps
    u = np.exp(np.sqrt(delta_t) * sigma)
    d = np.exp(-np.sqrt(delta_t) * sigma)
    p = (np.exp(delta_t * r) - d) / (u - d)

    n_rows = 2 * n_steps + 1
    n_rows_mid = (n_rows + 1) // 2
    tree = np.zeros((n_rows, n_steps + 1))
    tree[n_rows_mid - 1, 0] = S0

    for i in range(1, n_steps + 1):
        old_idx_non_zero = np.where(tree[:, i - 1] != 0)[0]
        new_idx_non_zero = np.unique(np.concatenate((old_idx_non_zero + 1, old_idx_non_zero - 1)))
        new_idx_non_zero = new_idx_non_zero[(new_idx_non_zero >= 0) & (new_idx_non_zero < n_rows)]
        new_S_val = np.unique(S0 * (u ** (new_idx_non_zero - (n_rows_mid - 1))))
        tree[new_idx_non_zero, i] = new_S_val

    x = np.arange(0, n_steps + 1)
    vec_prob_payoff = np.array([math.comb(n_steps, k) * (1 - p) ** k * p ** (n_steps - k) for k in x])
    bool_payoff = np.array([True if i % 2 == 0 else False for i in range(2 * n_steps)])
    bool_payoff = np.append(bool_payoff, True)

    vec_payoff = np.maximum(K - tree[bool_payoff, n_steps], 0)
    put_price_binomial_tree = np.exp(-T * r) * np.sum(vec_payoff * vec_prob_payoff)

    return {
        "mat_Binomial_Tree": tree,
        "vec_Payoff": vec_payoff,
        "vec_Prob_Payoff": vec_prob_payoff,
        "put_Price_Binomial_Tree": put_price_binomial_tree,
        "p": p,
        "d": d,
        "delta_t": delta_t
    }


In [13]:
put_binomial = black_scholes_put_binomial_tree(S0, K, r, sigma, T, n_steps)
call_binomial = black_scholes_call_binomial_tree(S0, K, r, sigma, T, n_steps)
print(f"Put Binomial = {put_binomial["put_Price_Binomial_Tree"]}")
print(f"Call Binomial = {call_binomial["call_Price_Binomial_Tree"]}")

Put Binomial = 0.13314135459567603
Call Binomial = 0.22698543790725767


In [14]:
print("Put price:", put_binomial["put_Price_Binomial_Tree"])
print("Probability:", put_binomial["vec_Prob_Payoff"])
print("Payoffs:", put_binomial["vec_Payoff"])
print("Matrix bonimial tree:\n", put_binomial["mat_Binomial_Tree"])

Put price: 0.13314135459567603
Probability: [0.03135459 0.15656348 0.31270861 0.31229083 0.1559368  0.03114569]
Payoffs: [0.63212056 0.45118836 0.18126925 0.         0.         0.        ]
Matrix bonimial tree:
 [[0.         0.         0.         0.         0.         0.36787944]
 [0.         0.         0.         0.         0.44932896 0.        ]
 [0.         0.         0.         0.54881164 0.         0.54881164]
 [0.         0.         0.67032005 0.         0.67032005 0.        ]
 [0.         0.81873075 0.         0.81873075 0.         0.81873075]
 [1.         0.         1.         0.         1.         0.        ]
 [0.         1.22140276 0.         1.22140276 0.         1.22140276]
 [0.         0.         1.4918247  0.         1.4918247  0.        ]
 [0.         0.         0.         1.8221188  0.         1.8221188 ]
 [0.         0.         0.         0.         2.22554093 0.        ]
 [0.         0.         0.         0.         0.         2.71828183]]


In [15]:
print("Call price:", call_binomial["call_Price_Binomial_Tree"])
print("Probability:", call_binomial["vec_Prob_Payoff"])
print("Payoffs:", call_binomial["vec_Payoff"])
print("Matrix bonimial tree:\n", call_binomial["mat_Binomial_Tree"])

Call price: 0.22698543790725767
Probability: [0.03135459 0.15656348 0.31270861 0.31229083 0.1559368  0.03114569]
Payoffs: [0.         0.         0.         0.22140276 0.8221188  1.71828183]
Matrix bonimial tree:
 [[0.         0.         0.         0.         0.         0.36787944]
 [0.         0.         0.         0.         0.44932896 0.        ]
 [0.         0.         0.         0.54881164 0.         0.54881164]
 [0.         0.         0.67032005 0.         0.67032005 0.        ]
 [0.         0.81873075 0.         0.81873075 0.         0.81873075]
 [1.         0.         1.         0.         1.         0.        ]
 [0.         1.22140276 0.         1.22140276 0.         1.22140276]
 [0.         0.         1.4918247  0.         1.4918247  0.        ]
 [0.         0.         0.         1.8221188  0.         1.8221188 ]
 [0.         0.         0.         0.         2.22554093 0.        ]
 [0.         0.         0.         0.         0.         2.71828183]]


## Analytic

# Black-Scholes Option Pricing Formulas

The price of a put and a call can be calculated using an analytical formula with the Black-Scholes model. The value of $S_0$ is the initial value of the asset price, the value of $K$ is the strike price. The value of $r$ is the risk-free interest rate and the value $\sigma$ is the volatility of the stock price. The maturity of the option in years is $T$. The formula for the put and call options are given by:

## Put Option

$$\text{Put} = e^{-rT}E\left[\max(K - S_T, 0)\right] = Ke^{-rT}\Phi(-d_2) - S_0\Phi(-d_1)$$

## Call Option

$$\text{Call} = e^{-rT}E\left[\max(S_T - K, 0)\right] = S_0\Phi(d_1) - Ke^{-rT}\Phi(d_2)$$

## Where

$$d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + 0.5\sigma^2\right)T}{\sigma\sqrt{T}}$$

$$d_2 = d_1 - \sigma\sqrt{T}$$

$$\Phi(x) = \int_{-\infty}^{x} \frac{1}{\sqrt{2\pi}} \exp(-0.5y^2) dy$$

Where $\Phi(x)$ is the cumulative distribution function of the standard normal distribution.

In [17]:
def black_Scholes_Call_Analytic(S_Zero, K, r, sigma, T):
    d1 = (np.log(S_Zero / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S_Zero * norm.cdf(d1) - K * np.exp(-r * (T)) * norm.cdf(d2)

def black_Scholes_Put_Analytic(S_Zero, K, r, sigma, T):
    d1 = (np.log(S_Zero / K) + (r + 0.5 * sigma ** 2) * (T)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * (T)) * norm.cdf(-d2) - S_Zero * norm.cdf(-d1)

In [20]:
S_Zero = 1

call_analytic = black_Scholes_Call_Analytic(S_Zero, K, r, sigma, T)
put_analytic = black_Scholes_Put_Analytic(S_Zero, K, r, sigma, T)
print(f"Put analytic = {put_analytic}")
print(f"Call analytic = {call_analytic}")

Put analytic = 0.1250582860086913
Call analytic = 0.22022086797273177
